互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  
    """计算二维互相关运算"""
    h,w = K.shape  # 获取卷积核的高度和宽度
    #X.shape[0] 是输入张量X的高度 ； X.shape[1] 是输入张量X的宽度
    #输出高度 = 输入高度 - 卷积核高度 + 1
    #输出宽度 = 输入宽度 - 卷积核宽度 + 1
    Y = torch.zeros((X.shape[0]- h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i + h, j:j +w] * K).sum()
    return Y    

验证上述二维互关运输的输出

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

实现二维卷积层

In [7]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        #定义两个可以学习参数
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积层的一个简单应用：检测图像中不同颜色的边缘

首先，我们构造一个6X8像素的黑白图像。中间四列为黑色（0），其余像素为白色（1）

In [8]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为1、宽度为2的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [9]:
K = torch.tensor([[1.0, -1.0]])

现在，我们对参数X（输入）和K（卷积核）执行互相关运算。 如下所示，输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为0

```
边缘检测原理
当卷积核滑过图像时，它计算的是相邻像素的差值：
情况1：从白到黑的边缘
    图像片段: [1, 0]  (白色到黑色)
    卷积运算: 1×1 + 0×(-1) = 1
    结果: 1 (检测到白→黑边缘)
情况2：从黑到白的边缘
    图像片段: [0, 1]  (黑色到白色)
    卷积运算: 0×1 + 1×(-1) = -1
    结果: -1 (检测到黑→白边缘)
情况3：颜色一致，无边缘
    图像片段: [1, 1] 或 [0, 0]  (颜色一致)
    卷积运算: 1×1 + 1×(-1) = 0 或 0×1 + 0×(-1) = 0
    结果: 0 (无边缘)

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核K只可以检测垂直边缘，无法检测水平边缘。

```
要检测水平边缘（垂直方向的颜色变化），需要垂直排列的卷积核：
pythonK_horizontal = torch.tensor([[1.0], 
                                  [-1.0]])  # 2×1 的垂直卷积核

In [11]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

学习由X生成Y的卷积核

```
Conv2d 参数解析
pythonnn.Conv2d(in_channels, out_channels, kernel_size, bias=False)
                       ↑           ↑            ↑          ↑
                     输入通道     输出通道      卷积核大小   是否使用偏置

In [12]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()  # 清空之前的梯度
    l.sum().backward()  # 计算梯度
    # 迭代卷积核
    #[:] 表示"所有元素"
    #更新参数
    #梯度下降公式：参数 = 参数 - 学习率 × 梯度
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

epoch 2, loss 12.927
epoch 4, loss 3.364
epoch 6, loss 1.054
epoch 8, loss 0.377
epoch 10, loss 0.145


现在我们来看看我们所学的卷积核的权重张量

In [13]:
# kernel_size=(1, 2)  # 这里的(1,2)是卷积核的尺寸
# h, w = K.shape      # h=1, w=2 是高度和宽度
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9468, -1.0238]])